# <img src="assets/ufscar.png" alt="Logo UFScar" width="200" align="left"/><p><center>Universidade Federal de São Carlos (UFSCar)</center><br/><font size="4"><center> Departamento de Computação, campus Sorocaba </center> </font>


<font size="4"><center><b>Disciplina: Aprendizado de Máquina</b></center></font>
<font size="3"><center>Prof. Dr. Tiago A. Almeida</center></font>
</p>

<br>
<font size = "4"><center><b> Grupo 5: Análise de sentimento de reviews na Amazon </b></center></font>

# Pré processamento: 
Funções do arquivo pre_processing.py

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pre_processing as pp
import analysis as anl
import pca
# Categoria da base de dados a ser lida (do disco) e processada
# [books, kitchen_&_housewares, electronics, dvd, all]
category = 'books'

# Se positivo, adiciona bigramas para reviews negativas
# ex: ('not', 'good') equivale a uma única feature
hNeg = True

# Se positivo, adiciona substantivos
noun = False

# Executa ou não o chi-quadrado na base
flagChi2 = True

# Guarda as features ja processadas em X, a classe da amostra em Y e o vocabulario em vocabulary
# hNeg e noun sao opcionais, por padrao hNeg=True, noun=False
X, Y, vocabulary = pp.bow(category, hNeg, noun)

print("Vocabulário possui " + str(len(vocabulary)) + " palavras!")

#### Separa os dados em treinamento e teste:

In [ ]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X2, Y2 = X[idx_perm, :], Y[idx_perm]

# Porcentagem de amostras destinadas à base de treino
pTrain = 0.8

# Executa o holdout e retorna os índices de treino e teste, mantendo a proporção original entre as classes
train_index, test_index = anl.stratified_holdOut(Y, pTrain)

# Guarda as amostras de treino e teste
Xtrain, Xval = X2[train_index, :], X2[test_index, :]
Ytrain, Yval = Y2[train_index], Y2[test_index]

#### Seleciona features com chi-quadrado (a partir dos dados de treinamento):

In [ ]:
# Seta o valor de alpha para o chi-quadrado. 
# alpha e opcional, por padrão alpha = 0.05
alpha = 0.05

# Chama a funcao para executar o chi-quadrado e retorna a nova base de dados reduzida
# o novo vocabulario e os indices das features mantidas
if (flagChi2):
    Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary, alpha)
    # Seleciona apenas as features do indice retornado pelo chi-quadrado para a base de teste
    Xval = Xval[:, index]

#converte para outro tipo de matriz esparsa que facilita os cálculos do svm
Xtrain = Xtrain.tolil() 
Xval = Xval.tolil()

In [ ]:
if (flagChi2):
    print("Número de features antes do chi-quadrado: " + str(len(vocabulary)))
    print("----------------------------------------")
    print("Número de features após chi-quadrado: " + str(len(new_vocabulary)))
    # print(new_vocabulary)

# Decision Tree Classifier

As árvores de decisão são algoritmos de aprendizado supervisionados usados para tarefas de classificação e regressão. No presente projeto, será usado apenas para classificação. As árvores de decisão são atribuídas aos algoritmos de aprendizado baseados em informações que usam diferentes medidas de ganho de informação para aprendizado. A idéia principal das árvores de decisão é encontrar as features que contêm mais "informações" no atributo alvo e, em seguida, dividir o conjunto de dados ao longo dos valores desses atributos, de modo que as características da classe de destino dos sub dataset resultantes sejam tão puras quanto quanto possível. Dessa forma, o conjuto é dividido recursivamente, com forme o ganho de informação, até chegar nos nós folhas, que representam a classe.

### Ganho de informação
O ganho de informação é a medida usada para dividir o conjuto de dados. Há várias formas de cálcular esse ganho: Índice de Gini, Entropia, Qui-Quadrado, Índice de Ganho de Informação, Variância. Será utilizado a entropia, nesse projeto.

### Entropia
A entropia de um conjunto de dados é usada para medir a impureza de um conjunto de dados. A fórmula da entropia: 
$$ Entropy(x) = - \sum_{k \in target} (p(x=k) * \log_2 p(x=k)$$

Onde p(x=k) é a probabidade de x ser igual a k. E k representa as classes do atributo alvo.

In [ ]:
import scipy.stats as st

#Foi utilizado o cálculo da entropia com a biblioteca scipy
def entropy(Y) :
    entropia = st.entropy(Y)
    return entropia


### Fórmula do ganho de informação:
Para poder verificar qual das featuers divide mais precisamente o conjunto de dados, ou seja, permanece o conjunto de dados com a impureza mais baixa - entropia ou, em outras palavras, classifica melhor a classe do atributo alvo, é feito da seguinte forma: Para cada feature, é feita a divisão do conjunto de dados ao longo dos valores dessas features e, em seguida, calculamos a entropia do conjunto de dados depois de dividir os dados ao longo dos valores da feature. Isso nos dá a entropia restante depois que é divido o conjunto de dados ao longo dos valores dessa feature. Em seguida, subtraímos esse valor da entropia originalmente calculada do conjunto de dados para ver quanto essa divisão de features reduz a entropia original. O ganho de informação de uma feature pode ser cálculada:

$$ InfoGain(feature_d)=Entropy(D)−Entropy(feature_d)$$


Fórmula para cálcular a Entropia por feature:

$$ InfoGain(feature_d, D) = Entropy(D) - \sum_{k \in target} \frac{| feature_d=k|}{|D|} * Entropia(feature_d=k) $$ 


In [ ]:
def infoGain(X, Y, split_atribute) :
    totalEntropy = entropy(Y) # entropia(D)
    valores, count = np.unique(X[:,split_atribute], return_counts = True)
    entropyPond = 0 # entropia poderada, entropia da feature split_atribute
    for i in range(len(valores)) : 
        
        myatt = entropy(np.where(X[:,split_atribute]==valores[i])[0]) # Entropia(X[feature_d==k])
        entropyPond += np.sum([(count[i]/np.sum(count)*myatt)]) # somatório de qtd(feature_d==k)/qtd(total) * Entropia(X[feature_d==k])
      
    informationGain =  totalEntropy - entropyPond
    
    return informationGain

### Algoritmo ID3
O algoritmo ID3 é um algoritmo popular para o crescimento de árvores de decisão, publicado por Ross Quinlan em 1986. Além do algoritmo ID3, há também outros algoritmos populares como o C4.5, o C5.0 e o CART. Será usado o ID3, por possuir a implementação mais simples.

O primeiro passo é defir os critérios de parada e como tratar cada um deles:

1 - Se todas as linhas do atributo alvo possuirem o mesmo valor:
    - Retorna a classe do atributo de alvo.
    
2 - O conjunto de dados não pode mais ser dividido, pois não há mais daos:
    - Será atribuido ao nó folha o valor do atributo alvo que ocorre com mais frequência no nó superior (pai). 
    
3 - O conjunto de dados não pode mais ser dividido, pois não tem mais features:
    - Será atribuido ao nó folha do atributo alvo o nó pai



Depois de definidos os critérios de parada, o algoritmo ID3 funciona de seguinte maneira:
- Escolher a feature que tem mais ganho de informação
- Para cada valor diferente na feature, adicione um nó sobre a raiz
- Para cada novo nó criar uma sub árvore chamando recursivamente o ID3

In [ ]:
def ID3(X, originalX, Y, originalY, features, parentNodeClasse = None) :
    # 1 - Se todas as linhas do atributo alvo possuirem o mesmo valor:
    if (len(np.unique(Y)) <= 1) :
        return np.unique(Y)[0] #retorna esse valor
    
    if (len(X)==0) : #O conjunto de dados estiver vazio:
        #o valor do atributo alvo que ocorre com mais frequência no nó superior (pai).
        return np.unique(originalY)[np.argmax(np.unique(originalY, return_counts = True))]
    
    if (len(features)==0) : # Se não houver mais feares
        return parentNodeClasse #retornar o atributo alvo do nó pai
    else :
        # classe do no pai, valor default -> classe majoritária em Y
        parentNodeClasse = np.unique(Y)[np.argmax(np.unique(Y,return_counts = True)[1])]
        bestGain = -330 #inicializando melhor ganho, -330, valor arbitrário negativo suficientemente pequeno
        #Procurar feature que o ferece melhor ganho
        bestFeature = features[0] #inicializando melhor feature
        for f in features :
            gain_new = infoGain(X,Y, f)
            if(gain_new > bestGain) : 
                bestGain = gain_new
                bestFeature = f
        #Estrutura da árvore        
        tree = {bestFeature:{}}
        
        #Tirar feature com melhor ganho do conjunto de feature
        #features.delete(bestFeature)
        features = [i for i in features if i != bestFeature]
        
        #Para cada valor na coluna da feature com melhor ganho
        for value in np.unique(X[:,bestFeature]) :
            value = value #evita um bug
            # dividindo os subconjunto
            subX_indes = np.where(X[:,bestFeature] == value)[0]
            subX = X[subX_indes]
            #subX e subY = seubConjunto onde feature escolhida possui o valor
            subY = Y[np.where(X[:,bestFeature] == value)]
            
            #chamada recursiva para ID3 -> crescer a árvore
            subTree = ID3(subX, X, subY, Y, features, parentNodeClasse)
            
            #colocar na estrutura da árvore 
            tree[bestFeature][value] = subTree
            
        return(tree) 
        

    

### Predição:
A predição ocorre de meneira recursiva, percorrendo a àrvore e verificando se chegou em um nó folha ou se ainda está em uma sub árvore.

#### Observação Importante
Desde que o modelo da árvore foi treinado e, em seguida, é mostrado ao modelo uma amostra que o modelo nunca havia visto, pode acontecer que os valores das features destas amostra não existam no modelo da árvore, porque não são as instâncias de treinamento tiveram esse valor para essa feature em específico. Isso causa uma exception e precisa ser tratada. Como este é um algoritmo simplificado, tratamos esse erro, retornando uma classe default, que nesse projeto, não é grave a predição incorreta. Como a maioria dos reviews na Amazon são positivos, a classe default escolhida foi a 1. 

In [ ]:
def predict(row, tree,default=1) :
    features = np.arange(Xtrain.shape[1])
    for f in features : #andar pelas features
        if f in list(tree.keys()) : #verificar se esta feature corresponde ao nó percorrido. No caso da primeira rodada, tem que ser a feature da raiz
            try : # referente a observação feita
                classe = tree[f][row[f]] #caso não tenha na árvore
            except:
                return default    # retorne o valor padrão
            
            classe = tree[f][row[f]] #caso tenha na árvore
            
            #verificar se é folha ou não
            if isinstance(classe,dict):
                return predict(row,classe) # se não for folha, chamar recursivamente a predição
            else:
                return classe # se for folha  retorna a classe
        

In [ ]:
def test(test, tree) :
    result = []
    for row in range(test.shape[0]) :
        t = test[row]
        result.append(predict(t, tree))
    return result    

In [ ]:
from pprint import pprint
features_index = np.arange(Xtrain.shape[1])

#treinamento
tree = ID3(Xtrain.toarray(), Xtrain.toarray() ,Ytrain, Ytrain, features_index)

#pprint(tree) #se quiser imprimir a árvore
result = test(Xval.toarray(), tree)

acuracia = np.sum(result==Yval)/len(Yval)
print(acuracia)

### Validação com K-Folds

In [ ]:
import k_folds as kf

#Pega todos os tipos de classes 
classes = classes = np.unique(Y)

# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X3, Y3 = X[idx_perm, :], Y[idx_perm]

# separa os dados em k folds
nFolds = 5
folds = kf.stratified_kfolds(Y3, nFolds, classes)

k = 1
resultados=[] # cria uma lista vazia para guardar os resultados obtidos em cada fold

for train_index, test_index in folds:

    print('\n-----------\n%d-fold: \n-----------\n' % (k) )

    # se train_index ou test_index forem vazios, interrompe o laco de repeticao
    if len(train_index)==0 or len(test_index)==0: 
        print('\tErro: o vetor com os indices de treinamento ou o vetor com os indices de teste esta vazio')      
        break
        
    Xtrain, Xval = X3[train_index, :], X3[test_index, :];
    Ytrain, Yval= Y3[train_index], Y3[test_index];

    if (flagChi2):
        Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
        Xval = Xval[:, index]
    
    #Converte matrizes esparsas para np arrays, para os cálculos da rede neural
    Xtrain = Xtrain.toarray()
    Xval = Xval.toarray()
    
    features_index = np.arange(Xtrain.shape[1])
    tree = ID3(Xtrain, Xtrain ,Ytrain, Ytrain, features_index)

    #pprint(tree)
    Ypred = test(Xval, tree)
    auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)

    # adiciona os resultados do fold atual na lista de resultados
    resultados.append( auxResults )
    
    k = k + 1
    
kf.mediaFolds( resultados, classes )